In [1]:
import os
import re
from pathlib import Path

import pandas as pd

In [2]:
from conflab.constants import (
    ff_raw_data_path, 
    ff_processed_data_path,
    vid3_offset_from_vid2)

In [3]:
s = '(<4,33>,cam2)(<9,26,27>,cam2)(<7,34>,cam2)(<8,10>,cam4)(<35,19,12>,cam4)(<21,13>,cam4)(<18,29,14,16>,cam4)(<6>,cam4)(<22,11>,cam6)(<32,15,17,2>,cam6)(<3,20>, cam6)(<23,25,28,24>,cam8)(<31,5>,cam8)(<30,1>,cam8)'

In [4]:
d = pd.read_csv(os.path.join(ff_raw_data_path, 'seg2.csv'), header=None)
d.head()

,0,1
0,14:00,"(<4,33>,cam2)(<9,26,27>,cam2)(<7,34>,cam2)(<8,..."
1,14:01,"(<4,33>,cam2)(<9,26,27>,cam2)(<7,34>,cam2)(<8,..."
2,14:02,"(<4,33>,cam2)(<9,26,27>,cam2)(<7,34>,cam2)(<8,..."
3,14:03,"(<4,33>,cam2)(<9,26,27>,cam2)(<7,34>,cam2)(<8,..."
4,14:04,"(<4,33>,cam2)(<9,26,27>,cam2)(<7,34>,cam2)(<8,..."


In [5]:
re.findall(r'\(<([0-9,]*)>, ?cam([0-9])\)', s)

[('4,33', '2'),
 ('9,26,27', '2'),
 ('7,34', '2'),
 ('8,10', '4'),
 ('35,19,12', '4'),
 ('21,13', '4'),
 ('18,29,14,16', '4'),
 ('6', '4'),
 ('22,11', '6'),
 ('32,15,17,2', '6'),
 ('3,20', '6'),
 ('23,25,28,24', '8'),
 ('31,5', '8'),
 ('30,1', '8')]

In [6]:
def read_raw_annot(fpath):
    d = pd.read_csv(fpath, header=None)
    groups = []
    for row in d[1]:
        frame_groups = []
        matches = re.findall(r'\(<([0-9,]*)>, ?cam([0-9])\)', row)
        for m in matches:
            cam = int(m[1])
            pids = [int(e) for e in m[0].split(',')]

            frame_groups.append({
                'cam': cam,
                'pids': pids
            })
        groups.append(frame_groups)
    return groups              

In [7]:
def to_groups(annot, offset=0):
    d = [
        {tuple(group_data['pids']): 
            {
                'processed': False,
                'cam': group_data['cam']
            } for group_data in frame_data
        } for frame_data in annot
    ]

    groups = []
    for f, frame_data in enumerate(d):
        for pids, info in frame_data.items():
            if info['processed'] == True: continue

            cam = d[f][pids]['cam']
            g = f
            while g < len(d) and pids in d[g] and d[g][pids]['processed'] == False:
                d[g][pids]['processed'] = True
                g += 1

            groups.append({
                'pids': pids,
                'start': f + offset,
                'end': g + offset,
                'cam': cam
            })

    return groups

In [8]:
annot2 = read_raw_annot(os.path.join(ff_raw_data_path, 'seg2.csv'))
annot3 = read_raw_annot(os.path.join(ff_raw_data_path, 'seg3.csv'))

In [9]:
len(annot2), len(annot3)

(219, 1058)

In [11]:
vid3_offset_from_vid2.total_seconds()

217.0

In [12]:
groups = to_groups(annot2) + to_groups(annot3[:12*60], offset=vid3_offset_from_vid2.total_seconds())

In [13]:
len(groups)

178

In [14]:
groups

[{'pids': (4, 33), 'start': 0, 'end': 219, 'cam': 2},
 {'pids': (9, 26, 27), 'start': 0, 'end': 46, 'cam': 2},
 {'pids': (7, 34), 'start': 0, 'end': 51, 'cam': 2},
 {'pids': (8, 10), 'start': 0, 'end': 22, 'cam': 4},
 {'pids': (35, 19, 12), 'start': 0, 'end': 219, 'cam': 4},
 {'pids': (21, 13), 'start': 0, 'end': 146, 'cam': 4},
 {'pids': (18, 29, 14, 16), 'start': 0, 'end': 183, 'cam': 4},
 {'pids': (6,), 'start': 0, 'end': 45, 'cam': 4},
 {'pids': (22, 11), 'start': 0, 'end': 219, 'cam': 6},
 {'pids': (32, 15, 17, 2), 'start': 0, 'end': 219, 'cam': 6},
 {'pids': (3, 20), 'start': 0, 'end': 51, 'cam': 6},
 {'pids': (23, 25, 28, 24), 'start': 0, 'end': 107, 'cam': 8},
 {'pids': (31, 5), 'start': 0, 'end': 120, 'cam': 8},
 {'pids': (30, 1), 'start': 0, 'end': 170, 'cam': 8},
 {'pids': (8,), 'start': 22, 'end': 45, 'cam': 4},
 {'pids': (10,), 'start': 22, 'end': 51, 'cam': 4},
 {'pids': (8, 6), 'start': 45, 'end': 219, 'cam': 4},
 {'pids': (26, 27), 'start': 46, 'end': 47, 'cam': 2},
 {'

In [15]:
pd.DataFrame(groups).to_csv(os.path.join(ff_processed_data_path, 'groups.csv'))